Section 1

1. (core) Animating fractal constructions gives us direct insight into how the function generating the fractal operates. You are given a pygame animation function draw_sierpinski that draws the Sierpinski triangle. Using this function the user is able to choose the depth of the triangle and to stop and start the animation. You should develop this function so that the user is also able to change the speed of the animation. You should also add colours to the triangle drawing.

In [3]:
pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -------------- ------------------------- 3.9/10.6 MB 23.8 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 30.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
